In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI
import os
import re
import time
import csv
from collections import Counter

In [3]:
df = pd.read_csv('CNC_synth_copy2.csv')

In [126]:
df.head()

,Text
0,The company accused <ARG1>the employee of stea...
1,"""<ARG1>The students organized a protest</ARG1>..."
2,'<ARG1>The students are excited</ARG1> <SIG0>b...
3,'<ARG0>Poor air quality</ARG0> <SIG0>can <ARG1...
4,Since the news about the company's financial d...


In [127]:
print(len(df))

1098


In [4]:
def filter_dataframe(df, word_one, word_two, word_three):
    filtered_df = df[df['Text'].str.contains(word_one, case=False) & df['Text'].str.contains(word_two, case=False)& df['Text'].str.contains(word_three, case=False)]

    return filtered_df

In [5]:
def filter_dataframe2(df, word_one, word_two, word_three, word_four, word_five, word_six):
    filtered_df = df[df['Text'].str.contains(word_one, case=False) & df['Text'].str.contains(word_two, case=False)& df['Text'].str.contains(word_three, case=False)& df['Text'].str.contains(word_four, case=False)& df['Text'].str.contains(word_five, case=False)& df['Text'].str.contains(word_six, case=False)]

    return filtered_df

In [6]:
def filter_dataframe_l(df, word_list):
    # Constructing the dynamic condition based on the list of words
    condition = df['Text'].apply(lambda x: all(re.search(re.escape(word), x, re.IGNORECASE) for word in word_list))

    # Applying the condition to filter the DataFrame
    filtered_df = df[condition]

    return filtered_df

In [7]:
def replace_double_spaces(df, column_name):
    # Ensure the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame.")

    # Replace double spaces with single spaces in the specified column
    df2 = df
    df2[column_name] = df[column_name].str.replace('  ', ' ')
    return df2

In [8]:
def process_text(text):
    # Characters to check for
    punctuation_chars = [',', '"', "'"]

    # Iterate through each character in the text
    processed_text = ""
    for char in text:
        # Add spaces around specified punctuation characters
        if char in punctuation_chars:
            processed_text += f' \{char} '
        else:
            processed_text += char
    text = processed_text
    start_index = 0
    end_index = len(text)

    # Find the index of the first letter, <, or >
    for i, char in enumerate(text):
        if char.isalpha() or char == '<' or char == '>'or text[i] == '.':
            start_index = i
            break

    # Find the index of the last letter, <, or >
    for i in range(len(text) - 1, -1, -1):
        if text[i].isalpha() or text[i] == '<' or text[i] == '>'or text[i] == '.':
            end_index = i + 1
            break

    # Extract the substring between the first and last letters
    result = text[start_index:end_index].strip()

    result += '.'
    result = "['{}']".format(result)

    return result

In [11]:
df['Text'] = df['Text'].apply(lambda x: process_text(x))

In [12]:
print(len(df))

1098


In [43]:
#this deletes any entries that lack an ARG0 or an ARG1
df_f = filter_dataframe(df,'<ARG0>', '<ARG1>','<SIG0>')

In [12]:
filter_list = ['<ARG0>','<ARG1>','<SIG0>','</ARG0>','</ARG1>','</SIG0>']
df_f = filter_dataframe_l(df, filter_list)

In [14]:
len(df_f)

808

In [15]:
df_f.head()

,Text
0,['The company accused <ARG1>the employee of st...
2,['<ARG1>The students are excited</ARG1> <SIG0>...
4,['Since the news about the company \' s financ...
5,"['On the morning of September 10th \, 2015 at..."
7,['She failed to water the plants <ARG0>for sev...


In [16]:
print(df_f['Text'].iloc[2])

['Since the news about the company \' s financial downfall spread \,  <ARG0>many employees</ARG0> <SIG0>have started to look for new job opportunities</SIG0> <ARG1>to secure their future.</ARG1>.']


In [36]:
def find_common_words(df):
    word_frequency = Counter()

    # Iterate through each entry in the DataFrame
    for index, row in df.iterrows():
        # Combine the entries into a single string
        combined_text = ' '.join(row)

        # Tokenize the text into words (you might need a more sophisticated tokenizer based on your data)
        words = re.findall(r'\b\w+\b', combined_text.lower())

        # Update word frequency based on the set of unique words in the entry
        word_frequency.update(set(words))

    # Get the most common words across all entries
    most_common_words = word_frequency.most_common()

    return most_common_words

In [18]:
#This is to get a statistic of the most common words used in each of the sentences. This will inform which entries will be cut from the final dataset.

common_words = find_common_words(df_f)
print(common_words)

[('arg1', 808), ('arg0', 808), ('sig0', 808), ('the', 715), ('to', 563), ('in', 456), ('and', 307), ('a', 300), ('of', 269), ('heavy', 231), ('due', 185), ('city', 145), ('flooding', 137), ('caused', 133), ('rain', 117), ('widespread', 99), ('resulting', 98), ('their', 97), ('for', 95), ('rainfall', 94), ('company', 87), ('new', 79), ('because', 77), ('after', 76), ('led', 75), ('increase', 74), ('s', 70), ('as', 70), ('on', 68), ('government', 66), ('damage', 65), ('was', 60), ('students', 59), ('has', 58), ('causing', 56), ('were', 56), ('streets', 56), ('by', 55), ('that', 53), ('severe', 52), ('flooded', 52), ('traffic', 50), ('leading', 50), ('order', 45), ('levels', 45), ('lack', 45), ('increased', 44), ('homes', 42), ('several', 40), ('pollution', 40), ('infrastructure', 40), ('residents', 40), ('resulted', 39), ('many', 38), ('rise', 38), ('implemented', 38), ('she', 37), ('with', 37), ('decided', 35), ('significant', 35), ('decrease', 35), ('car', 34), ('protest', 33), ('have'

In [15]:
csv_file_path = 'CNC_train_raw.csv'
df_f.to_csv(csv_file_path, index=False)

In [252]:
#code block by shubhika


# Adding a new column 'sent_id' starting from 10000 and incrementing by 1
df_f['sent_id'] = range(10000, 10000 + len(df_f))

# Adding a new column 'corpus' with constant value 'cnc'
df_f['corpus'] = 'cnc'

# Adding a new column 'eg_id' with constant value 0
df_f['eg_id'] = 0

# Adding a new column 'doc_id' with values 'train_1000_10000' where 'train_' is constant, and the other two values are incremented by 1
df_f['doc_id'] = [f'train_{i}_{j}' for i, j in zip(range(1000, 1000 + len(df_f)), range(10000, 10000 + len(df_f)))]

# Adding a new column 'index' with values based on variables i, j, k
df_f['index'] = [f'cnc_train_{i}_{j}_{k}_10' for i, j, k in zip(range(100, 100 + len(df_f)), range(1000, 1000 + len(df_f)), range(10000, 10000 + len(df_f)))]


/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_80111/771670798.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['sent_id'] = range(10000, 10000 + len(df_f))
/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_80111/771670798.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['corpus'] = 'cnc'
/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_80111/771670798.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [253]:
df_f.rename(columns={'Text': 'causal_text_w_pairs'}, inplace=True)

/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_80111/3188732107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f.rename(columns={'Text': 'causal_text_w_pairs'}, inplace=True)


In [254]:
df_f2 = df_f

In [255]:
df_f2['text'] = df_f2['causal_text_w_pairs'].apply(lambda x: re.sub(r'<(/?ARG[01]|/?SIG0)>', '', x))

/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_80111/1233164403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f2['text'] = df_f2['causal_text_w_pairs'].apply(lambda x: re.sub(r'<(/?ARG[01]|/?SIG0)>', '', x))


In [256]:
#Always 1 because the data is based on only 1 causal relationship sentences.
df_f2['num_rs'] = 1

/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_80111/280037877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f2['num_rs'] = 1


In [257]:
#orders the field names, also based on the code from shubhika
df_f2 = df_f2[['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text_w_pairs', 'num_rs']]

In [9]:
def remove_wrapping(text):
    if isinstance(text, str) and text.startswith("['") and text.endswith("']"):
        return text[2:-2]  # Remove the first two and last two characters
    else:
        return text

In [558]:
df_f2['text'] = df_f2['text'].apply(lambda x: process_text(x))

In [559]:
df_f2['text'] = df_f2['text'].apply(remove_wrapping)

In [585]:
df_f2 = df_f2[~df_f2['causal_text_w_pairs'].str.contains('<SIG1>')].copy()
df_f2.reset_index(drop=True, inplace=True)

In [586]:
df_f2.to_csv('CNC_chatgpt_output14.csv', index=None)

In [10]:
df_o = pd.read_csv('CausalNewsCorpus/data/V2/train_subtask2_grouped.csv')

In [587]:
result_df = pd.concat([df_o, df_f2], ignore_index=True)

In [583]:
len(result_df)

3883

In [584]:
len(df_o)

3075

In [34]:
filter_list3 = ['<ARG0>','<ARG1>','</ARG0>','</ARG1>']
df_ff = filter_dataframe_l(df, filter_list3)
# Adding a new column 'sent_id' starting from 10000 and incrementing by 1
df_ff['sent_id'] = range(10000, 10000 + len(df_ff))

# Adding a new column 'corpus' with constant value 'cnc'
df_ff['corpus'] = 'cnc'

# Adding a new column 'eg_id' with constant value 0
df_ff['eg_id'] = 0

# Adding a new column 'doc_id' with values 'train_1000_10000' where 'train_' is constant, and the other two values are incremented by 1
df_ff['doc_id'] = [f'train_{i}_{j}' for i, j in zip(range(1000, 1000 + len(df_ff)), range(10000, 10000 + len(df_ff)))]

# Adding a new column 'index' with values based on variables i, j, k
df_ff['index'] = [f'cnc_train_{i}_{j}_{k}_10' for i, j, k in zip(range(100, 100 + len(df_ff)), range(1000, 1000 + len(df_ff)), range(10000, 10000 + len(df_ff)))]
df_ff.rename(columns={'Text': 'causal_text_w_pairs'}, inplace=True)
df_ff2 = df_ff
df_ff2['text'] = df_ff2['causal_text_w_pairs'].apply(lambda x: re.sub(r'<(/?ARG[01]|/?SIG0)>', '', x))

df_ff2['num_rs'] = 1
df_ff2 = df_ff2[['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text_w_pairs', 'num_rs']]
df_ff2['text'] = df_ff2['text'].apply(lambda x: process_text(x))
df_ff2['text'] = df_ff2['text'].apply(remove_wrapping)
df_ff2['text'] = df_ff2['text'].str.replace(r'\s{2,}', ' ', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(r'\s{2,}', ' ', regex=True)
df_ff2['text'] = df_ff2['text'].str.replace('> ', '>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('> ', '>', regex=True)
df_ff2['text'] = df_ff2['text'].str.replace('><', '> <', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('><', '> <', regex=True)
df_ff2 = df_ff2[~df_ff2['causal_text_w_pairs'].str.contains('<SIG1>')].copy()
df_ff2.reset_index(drop=True, inplace=True)

df_f3 = df_ff2[~df_ff2['causal_text_w_pairs'].str.contains('SIG0')].copy()
df_f3.reset_index(drop=True, inplace=True)
print(len(df_f3))
result_df2 = pd.concat([df_o, df_f3], ignore_index=True)
#result_df2.to_csv('CNC_chatgpt+orig_output_no_sig6.csv', index=None)

114


/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_33444/3421329401.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ff['sent_id'] = range(10000, 10000 + len(df_ff))
/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_33444/3421329401.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ff['corpus'] = 'cnc'
/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_33444/3421329401.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [588]:
result_df.to_csv('CNC_chatgpt+orig_output10.csv', index=None)

In [39]:
print(df_ff2['causal_text_w_pairs'].iloc[25])

['<ARG1>Due to heavy rainfall and flooding \, the school was forced to cancel classes for the day</ARG1>because <ARG0>the classrooms were deemed unsafe for students</ARG0>..']


In [600]:
df_f3.to_csv('no_sig_chatGPT2.csv', index = None)

In [147]:
df_o['causal_text_w_pairs'].head()

0    ['The State alleged <ARG1>they hacked Sabata P...
1                                                   []
2    ["<ARG1>The farmworkers ' strike resumed on Tu...
3    ['<ARG1>Demonstrators have filed for a permit<...
4    ['<ARG0>Footage of the attack , which included...
Name: causal_text_w_pairs, dtype: object

In [42]:
df_ff = filter_dataframe_l(df, filter_list)
df_ff = df_ff[~df_ff['Text'].str.contains('</ARG0><SIG0>')].copy()
df_ff = df_ff[~df_ff['Text'].str.contains('</ARG0></SIG0>')].copy()
df_ff = df_ff[~df_ff['Text'].str.contains('</ARG1><SIG0>')].copy()
print(len(df_ff))

794


In [13]:

df_ff = filter_dataframe_l(df, filter_list)
df_ff = df_ff[~df_ff['Text'].str.contains('</ARG0><SIG0>')].copy()
df_ff = df_ff[~df_ff['Text'].str.contains('</ARG0></SIG0>')].copy()
df_ff = df_ff[~df_ff['Text'].str.contains('</ARG1><SIG0>')].copy()
# Adding a new column 'sent_id' starting from 10000 and incrementing by 1
df_ff['sent_id'] = range(10000, 10000 + len(df_ff))

# Adding a new column 'corpus' with constant value 'cnc'
df_ff['corpus'] = 'cnc'

# Adding a new column 'eg_id' with constant value 0
df_ff['eg_id'] = 0

# Adding a new column 'doc_id' with values 'train_1000_10000' where 'train_' is constant, and the other two values are incremented by 1
df_ff['doc_id'] = [f'train_{i}_{j}' for i, j in zip(range(1000, 1000 + len(df_ff)), range(10000, 10000 + len(df_ff)))]

# Adding a new column 'index' with values based on variables i, j, k
df_ff['index'] = [f'cnc_train_{i}_{j}_{k}_10' for i, j, k in zip(range(100, 100 + len(df_ff)), range(1000, 1000 + len(df_ff)), range(10000, 10000 + len(df_ff)))]
df_ff.rename(columns={'Text': 'causal_text_w_pairs'}, inplace=True)
df_ff2 = df_ff
df_ff2['text'] = df_ff2['causal_text_w_pairs'].apply(lambda x: re.sub(r'<(/?ARG[01]|/?SIG0)>', '', x))

df_ff2['num_rs'] = 1
df_ff2 = df_ff2[['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text_w_pairs', 'num_rs']]
df_ff2['text'] = df_ff2['text'].apply(lambda x: process_text(x))
df_ff2['text'] = df_ff2['text'].apply(remove_wrapping)
df_ff2['text'] = df_ff2['text'].str.replace(r'\s{2,}', ' ', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(r'\s{2,}', ' ', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('> ', '>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('><', '> <', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('</SIG0>', '</SIG0> ', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(' </ARG0> </SIG0>', '</ARG0> </SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(' </ARG0> <SIG0>', '</ARG0> <SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(' </ARG1> <SIG0>', '</ARG1> <SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('<ARG0> <SIG0>', '<ARG0><SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('<ARG1> <SIG0>', '<ARG1><SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(' </ARG0> <ARG1><SIG0>', '</ARG0> <ARG1><SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(' </ARG0> <ARG1></SIG0>', '</ARG0> <ARG1></SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(' </ARG0> </ARG1></SIG0>', '</ARG0> </ARG1></SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('</ARG0>', '</ARG0> ', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('</ARG1>', '</ARG1> ', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace('</ARG1> <SIG0>', '</ARG1><SIG0>', regex=True)
df_ff2['causal_text_w_pairs'] = df_ff2['causal_text_w_pairs'].str.replace(' </ARG1> <ARG0><SIG0>', '</ARG1> <ARG0><SIG0>', regex=True)
print(len(df_ff2))
result_df2 = pd.concat([df_o, df_ff2], ignore_index=True)
#result_df2.to_csv('CNC_chatgpt+orig_output_no_space6.csv', index=None)

794


In [150]:
type(df_o['causal_text_w_pairs'].iloc[0])

str

In [151]:
print(df_o['causal_text_w_pairs'].iloc[0])

['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .']


In [278]:
def replace_tokens(df, column_name):
    # Define a mapping for replacement
    replacement_mapping = {
        '<arg0>': '<ARG0>',
        '<arg1>': '<ARG1>',
        '<sig0>': '<SIG0>',
        '</arg0>': '</ARG0>',
        '</arg1>': '</ARG1>',
        '</sig0>': '</SIG0>'
    }

    # Function to replace tokens in a text
    def replace_in_text(text):
        for token, replacement in replacement_mapping.items():
            text = text.replace(token, replacement)
        return text

    # Apply the function to the specified column
    df[column_name] = df[column_name].apply(replace_in_text)
    return df


In [491]:
def remove_non_bracket_chars(text):
    # Extract characters between brackets using a regular expression
    #text = text.replace('[\'', '').replace(']\'', '')
    #extracted_chars = re.findall(r'\[([^\]]*)\]', text)
    result = text.replace('[', '').replace(']', '')
    # Concatenate the extracted characters to form the result
    result = '[' + result + ']'
    return result

In [537]:
def remove_non_quoted_chars(text):
    #in_single_quote = False
    result = text.replace('[', '').replace(']', '')
    '''
    for char in result:
        if char == ' ' and in_single_quote:
            result += "\'"
            in_single_quote = False
        elif char == "'":
            in_single_quote = True
            result += "\'"
        else:
            result += char
    in_single_quote = False
    for char in result:
        if char == ' ' and in_single_quote:
            result += '\"'
            in_single_quote = False
        elif char == '"':
            in_single_quote = True
            result += '\"'
        else:
            result += char
    '''
    result.replace(" ' ", "@z")
    pattern = r'\'(.*?)\'|\"(.*?)\"'
    matches = re.findall(pattern, result)
    result_list = [f"'{match[0]}'" if match[0] else f'"{match[1]}"' for match in matches]
    result_f = ', '.join(result_list)
    result_f = '[' + result_f + ']'
    result_f.replace("@z", " ' ")
    return result_f

In [568]:
df_eda = pd.read_csv('eda_2_output_transformed_train_subtask2.csv')

In [569]:
df_eda = df_eda[~df_eda['causal_text_w_pairs'].str.contains('\'s', case=False, na=False)]
df_eda = df_eda[~df_eda['causal_text_w_pairs'].str.contains(' \' ', case=False, na=False)]

In [570]:
df_eda['text'] = df_eda['text'].apply(remove_wrapping)

In [571]:
df_eda = replace_tokens(df_eda,'causal_text_w_pairs')

In [572]:
df_eda['causal_text_w_pairs'] = df_eda['causal_text_w_pairs'].apply(remove_non_bracket_chars)

In [573]:
df_eda['causal_text_w_pairs'] = df_eda['causal_text_w_pairs'].apply(remove_non_quoted_chars)

In [552]:
df_eda.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_1000_10000,10000,0,cnc_train_100_1000_10000_10,'the say alleged they hacked sabata petros cha...,['the say alleged <ARG1>they hacked sabata pet...,1
1,cnc,train_1001_10001,10001,0,cnc_train_101_1001_10001_10,'the state aver they hacked sabata petros chal...,['the state aver <ARG1>they hacked sabata petr...,1
2,cnc,train_1002_10002,10002,0,cnc_train_102_1002_10002_10,'the state alleged they hacked sabata petros c...,['the state alleged <ARG1>they hacked sabata p...,1
3,cnc,train_1003_10003,10003,0,cnc_train_103_1003_10003_10,'the state alleged they hacked sabata petros c...,['the state alleged <ARG1>they hacked sabata p...,1
4,cnc,train_1004_10004,10004,0,cnc_train_104_1004_10004_10,'the state alleged they hacked sabata petros c...,['the state alleged <ARG1>they hacked sabata p...,1


In [554]:
eda_result_df = pd.concat([df_o, df_eda], ignore_index=True)
eda_result_df.to_csv('CNC_eda_o+orig_output11.csv', index=None)

In [504]:
df_eda.to_csv('testing_func10.csv',index=None)

In [431]:
def check_words_in_column(df, column_name, word_list):
    """
    Check if every entry in a DataFrame column contains every word in the inputted list.

    Parameters:
    - df: DataFrame
    - column_name: str, the name of the column in the DataFrame
    - word_list: list of str, the list of words to check for in each entry

    Returns:
    - result: bool, True if every entry contains every word, False otherwise
    """

    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Iterate through each row in the specified column
    for entry in df[column_name]:
        # Check if every word in the word_list is present in the entry
        if all(word.lower() in entry.lower() for word in word_list):
            continue
        else:
            print(entry)
            return False

    # If the loop completes without returning False, all entries pass the check
    return True

In [553]:
filter_list2 = ['<ARG0>','<ARG1>','</ARG0>','</ARG1>']
print(check_words_in_column(df_eda, 'causal_text_w_pairs', filter_list2))

True


In [555]:
len(df_eda)

6775

In [556]:
df_eda_p = pd.read_csv('processed3_eda_3_output_transformed_train_subtask2-2.csv')
eda_p_result_df = pd.concat([df_o, df_eda_p], ignore_index=True)
eda_p_result_df.to_csv('CNC_eda_p_o+orig_output.csv', index=None)

In [557]:
print(check_words_in_column(df_eda_p, 'causal_text_w_pairs', filter_list2))

['on th , subgenus chen daoxiang , the headspring of the taiwanese united states army fort in hong kong , enunciate <ARG1>the military machine was ascertain to protect [ the ] ']
False


In [577]:
df_t = df_eda[df_eda['num_rs'] == 1].copy()
df_t.reset_index(drop=True, inplace=True)
df_t = df_t[~df_t['causal_text_w_pairs'].str.contains('<sig1>')].copy()
df_t.reset_index(drop=True, inplace=True)

In [578]:
print(check_words_in_column(df_t, 'causal_text_w_pairs', filter_list2))

True


In [579]:
eda_result_t_df = pd.concat([df_o, df_t], ignore_index=True)
eda_result_t_df.to_csv('CNC_eda_o+orig_output55.csv', index=None)

In [590]:
print(check_words_in_column(df_f2, 'causal_text_w_pairs', filter_list2))

True


In [591]:
df_t2 = pd.read_csv('del_4_processed_eda_2_output_transformed_train_subtask2.csv')
print(check_words_in_column(df_t2, 'causal_text_w_pairs', filter_list2))

True


In [592]:
df_t3 = pd.read_csv('del_4_processed_eda_3_output_transformed_train_subtask2.csv')
print(check_words_in_column(df_t3, 'causal_text_w_pairs', filter_list2))

True


In [38]:
common_words = find_common_words(df_ff2[['causal_text_w_pairs']])
print(common_words)

[('arg1', 794), ('sig0', 794), ('arg0', 794), ('the', 701), ('to', 554), ('in', 450), ('and', 304), ('a', 296), ('of', 263), ('heavy', 227), ('due', 181), ('city', 144), ('flooding', 134), ('caused', 130), ('rain', 114), ('widespread', 98), ('resulting', 98), ('their', 96), ('for', 95), ('rainfall', 92), ('company', 86), ('new', 78), ('because', 76), ('after', 75), ('led', 74), ('increase', 71), ('as', 70), ('s', 69), ('on', 67), ('government', 65), ('damage', 64), ('students', 59), ('has', 57), ('causing', 56), ('was', 56), ('were', 55), ('streets', 55), ('by', 53), ('that', 52), ('flooded', 52), ('severe', 51), ('traffic', 50), ('leading', 50), ('levels', 45), ('order', 44), ('increased', 44), ('homes', 42), ('lack', 42), ('pollution', 40), ('infrastructure', 40), ('residents', 40), ('several', 39), ('many', 38), ('rise', 38), ('resulted', 38), ('implemented', 37), ('she', 36), ('with', 36), ('decided', 35), ('decrease', 35), ('significant', 34), ('car', 33), ('protest', 33), ('have'

In [30]:
df_no_sig = pd.read_csv('CNC_chatgpt+orig_output_no_sig65.csv')
df_EDA = pd.read_csv('CNC_eda_o+orig_output55.csv')
df_chatgpt = pd.read_csv('CNC_chatgpt+orig_output_no_space6.csv')

In [18]:
df_EDA.iloc[3075]

corpus                                                               cnc
doc_id                                                  train_1000_10000
sent_id                                                            10000
eg_id                                                                  0
index                                        cnc_train_100_1000_10000_10
text                   'the say alleged they hacked sabata petros cha...
causal_text_w_pairs    ['the say alleged <ARG1>they hacked sabata pet...
num_rs                                                                 1
Name: 3075, dtype: object

In [31]:
sub_df1 = df_EDA.iloc[3075:].copy()
sub_df3 = df_chatgpt.iloc[3075:].copy()
sub_df1.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
3075,cnc,train_1000_10000,10000,0,cnc_train_100_1000_10000_10,'the say alleged they hacked sabata petros cha...,['the say alleged <ARG1>they hacked sabata pet...,1
3076,cnc,train_1001_10001,10001,0,cnc_train_101_1001_10001_10,'the state aver they hacked sabata petros chal...,['the state aver <ARG1>they hacked sabata petr...,1
3077,cnc,train_1002_10002,10002,0,cnc_train_102_1002_10002_10,'the state alleged they hacked sabata petros c...,['the state alleged <ARG1>they hacked sabata p...,1
3078,cnc,train_1003_10003,10003,0,cnc_train_103_1003_10003_10,'the state alleged they hacked sabata petros c...,['the state alleged <ARG1>they hacked sabata p...,1
3079,cnc,train_1004_10004,10004,0,cnc_train_104_1004_10004_10,'the state alleged they hacked sabata petros c...,['the state alleged <ARG1>they hacked sabata p...,1


In [39]:
print(len(sub_df1))

4710


In [40]:
print(len(sub_df2))

114


In [41]:
print(len(sub_df3))

794


In [20]:
sub_df2 = df_no_sig.iloc[3075:].copy()
sub_df2.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
3075,cnc,train_1001_10001,10001,0,cnc_train_101_1001_10001_10,The students organized a protest because they ...,['<ARG1>The students organized a protest</ARG1...,1
3076,cnc,train_1009_10009,10009,0,cnc_train_109_1009_10009_10,The increase in tuition fees and lack of finan...,['The increase in tuition fees and lack of fin...,1
3077,cnc,train_1025_10025,10025,0,cnc_train_125_1025_10025_10,"Due to heavy rainfall and flooding \ \, the sc...","['<ARG1>Due to heavy rainfall and flooding \, ...",1
3078,cnc,train_1048_10048,10048,0,cnc_train_148_1048_10048_10,MUMBAI : High pollution levels : residents dem...,['MUMBAI : <ARG0>High pollution levels</ARG0> ...,1
3079,cnc,train_1056_10056,10056,0,cnc_train_156_1056_10056_10,"MUMBAI Water shortage worsens \ \, residents p...",['MUMBAI <ARG1>Water shortage worsens</ARG1> \...,1


In [32]:
com_df = pd.concat([sub_df3,sub_df1,sub_df2], ignore_index=True)

In [25]:
df_ff2.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_1000_10000,10000,0,cnc_train_100_1000_10000_10,The company accused the employee of stealing c...,The company accused <ARG1>the employee of stea...,1
2,cnc,train_1001_10001,10001,0,cnc_train_101_1001_10001_10,The students are excited because they won the ...,'<ARG1>The students are excited</ARG1> <SIG0>...,1
4,cnc,train_1002_10002,10002,0,cnc_train_102_1002_10002_10,Since the news about the company \' s financia...,Since the news about the company's financial d...,1
5,cnc,train_1003_10003,10003,0,cnc_train_103_1003_10003_10,"On the morning of September 10th \, 2015 at 3:...","On the morning of September 10th, 2015 at 3:49...",1
7,cnc,train_1004_10004,10004,0,cnc_train_104_1004_10004_10,She failed to water the plants for several wee...,'She failed to water the plants <ARG0>for seve...,1


In [24]:
com_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_1000_10000,10000,0,cnc_train_100_1000_10000_10,The company accused the employee of stealing c...,The company accused <ARG1>the employee of stea...,1
1,cnc,train_1001_10001,10001,0,cnc_train_101_1001_10001_10,The students are excited because they won the ...,'<ARG1>The students are excited</ARG1> <SIG0>...,1
2,cnc,train_1002_10002,10002,0,cnc_train_102_1002_10002_10,Since the news about the company \' s financia...,Since the news about the company's financial d...,1
3,cnc,train_1003_10003,10003,0,cnc_train_103_1003_10003_10,"On the morning of September 10th \, 2015 at 3:...","On the morning of September 10th, 2015 at 3:49...",1
4,cnc,train_1004_10004,10004,0,cnc_train_104_1004_10004_10,She failed to water the plants for several wee...,'She failed to water the plants <ARG0>for seve...,1


In [33]:
com_df['sent_id'] = range(10000, 10000 + len(com_df))
com_df['doc_id'] = [f'train_{i}_{j}' for i, j in zip(range(1000, 1000 + len(com_df)), range(10000, 10000 + len(com_df)))]

# Adding a new column 'index' with values based on variables i, j, k
com_df['index'] = [f'cnc_train_{i}_{j}_{k}_10' for i, j, k in zip(range(100, 100 + len(com_df)), range(1000, 1000 + len(com_df)), range(10000, 10000 + len(com_df)))]


In [34]:
final_result = pd.concat([df_o, com_df], ignore_index=True)
final_result.to_csv('EDA+CNC_chatgpt+orig2.csv', index=None)